In [49]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

In [50]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)

In [51]:
N_EPOCHS = 30
N_TRIALS = 20
BATCH_SIZE = 128

CONFIG = config_tests["random"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [52]:
def load_cifar_data2(filepath):
    x = []
    y = []

    batches = ["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5"]
    for batch in batches:
        data = utils.unpickle(filepath + batch)

        x_batch = data[b"data"].reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float32")
        y_batch = np.array(data[b"labels"])

        x.append(x_batch)
        y.append(y_batch)

    return np.array(x).reshape(-1, 32, 32, 3), np.array(y).reshape(-1)


In [53]:
x, y = load_cifar_data2("../data/cifar-10-batches-py/")

In [54]:
# filepath_prefix = "../data/data2/seg_train/seg_train/"
#
# buildings = load_class_data(f"{filepath_prefix}buildings")
# street = load_class_data(f"{filepath_prefix}street")
# forest = load_class_data(f"{filepath_prefix}forest")
# glacier = load_class_data(f"{filepath_prefix}glacier")
# sea = load_class_data(f"{filepath_prefix}sea")
# mountain = load_class_data(f"{filepath_prefix}mountain")

In [55]:
# x = np.concatenate([buildings, street, forest, glacier, sea, mountain])
# y = np.concatenate(
#     [
#         [0] * buildings.shape[0],
#         [1] * street.shape[0],
#         [2] * forest.shape[0],
#         [3] * glacier.shape[0],
#         [4] * sea.shape[0],
#         [5] * mountain.shape[0],
#     ]
# )

In [56]:
x /= 255

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]

In [57]:
x.shape

(50000, 32, 32, 3)

In [58]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

assessment_model.fit(x_train, y_train, epochs=30, batch_size=BATCH_SIZE)

Epoch 1/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.1968 - loss: 2.2730
Epoch 2/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.3558 - loss: 1.7349
Epoch 3/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.4153 - loss: 1.5788
Epoch 4/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.4495 - loss: 1.5071
Epoch 5/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.4757 - loss: 1.4229
Epoch 6/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.4965 - loss: 1.3842
Epoch 7/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.5140 - loss: 1.3409
Epoch 8/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.5262 - loss: 1.3089
Epoch 9/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.5384 - loss: 1.2806
Epoch 10/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.5499 - loss: 1.2464
Epoch 11/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.5588 - loss: 1.2391
Epoch 12/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 

In [59]:
assessment_model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6925 - loss: 0.9001


[0.9137845039367676, 0.6801999807357788]

In [9]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

samples_proba = utils.calculate_proba(
    assessment_model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
)

In [10]:
results = {}
# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]


for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=n_classes, **models_hyperparameters[test_model])
        model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)

            utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])
            samples_ids = np.random.choice(range(train_size), size=n_samples, replace=False)

            model.fit(
                x_train_sorted[samples_ids],
                y_train_sorted[samples_ids],
                validation_data=(x_val, y_val),
                epochs=1,
                batch_size=BATCH_SIZE,
                verbose=0,
            )
            
            if CONFIG["progressive"]:
                samples_proba = utils.calculate_proba(
                    model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
                )

        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)

        model_scores.append(accuracy)

    results[test_model] = model_scores

test_model_1


  0%|          | 0/20 [00:00<?, ?it/s]

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5283 - loss: 1.2764


  5%|▌         | 1/20 [00:58<18:30, 58.47s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5019 - loss: 1.3832


...

 90%|█████████ | 18/20 [17:35<01:52, 56.35s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4722 - loss: 1.3705


 95%|█████████▌| 19/20 [18:30<00:56, 56.12s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4740 - loss: 1.6082


100%|██████████| 20/20 [19:28<00:00, 58.41s/it]


In [11]:
results

{'test_model_1': [0.5270000100135803,
  0.5099999904632568,
  0.39899998903274536,
  0.4440000057220459,
  0.4359999895095825,
  0.4180000126361847,
  0.47200000286102295,
  0.4830000102519989,
  0.47099998593330383,
  0.5019999742507935,
  0.5040000081062317,
  0.44600000977516174,
  0.5149999856948853,
  0.4740000069141388,
  0.5260000228881836,
  0.2980000078678131,
  0.46299999952316284,
  0.43700000643730164,
  0.47999998927116394,
  0.46299999952316284]}

In [12]:
sum(results["test_model_1"]) / N_TRIALS

0.46340000033378603

In [13]:
rs1 = results["test_model_1"][:]

In [16]:
rs2 = [0.4189999997615814,
  0.4740000069141388,
  0.4740000069141388,
  0.46299999952316284,
  0.5270000100135803,
  0.49900001287460327,
  0.4560000002384186,
  0.49399998784065247,
  0.5189999938011169,
  0.4580000042915344,
  0.4819999933242798,
  0.4690000116825104,
  0.4440000057220459,
  0.4790000021457672,
  0.4569999873638153,
  0.5270000100135803,
  0.5090000033378601,
  0.5080000162124634,
  0.4699999988079071,
  0.46700000762939453]

In [17]:
# rs2 = results["test_model_1"][:]

In [18]:
stats.ttest_ind(rs1, rs2)

TtestResult(statistic=-1.2221055318053675, pvalue=0.22919454439533676, df=38.0)